In [4]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from scipy.io import loadmat
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import h5py

In [7]:
#Load .mat

mat = loadmat('C:/Users/aless/Documents/Paper/Calibration/OptionData_1996_2014_only_Wed.mat')

In [8]:
# Code for reading the names of the items in the .mat file

variables = mat.items()

for var in variables:
    name = var[0]
    data = var[1]
    print( "Name ", name)  # Name
    if type(data) is h5py.Dataset:
        # If DataSet pull the associated Data
        # If not a dataset, you may need to access the element sub-items
        value = data.value
        print("Value", value)  # NumPy Array / Value

Name  __header__
Name  __version__
Name  __globals__
Name  Option_Date
Name  Security_Date
Name  Option_Maturity
Name  Option_Strike
Name  Option_Type
Name  Option_Price
Name  Security_Price
Name  Option_Volume
Name  Option_IV
Name  ind_wed
Name  PWed
Name  ind_n


In [9]:
Option_Strike = mat['Option_Strike']
Option_Maturity = mat['Option_Maturity']
Security_Strike = mat['Security_Price']
Option_Price = mat['Option_Price']

In [10]:
Option_Maturity

array([[ 66,  59,  52, ...,   3, 360, 353],
       [528, 521, 514, ...,   3, 360, 353],
       [ 38,  31, 150, ...,   3, 360, 353],
       ...,
       [  0,   0,   0, ...,   0,  79,   0],
       [  0,   0,   0, ...,   0,  79,   0],
       [  0,   0,   0, ...,   0,  79,   0]], dtype=uint16)

In [11]:
Option_Strike

array([[ 600,  600,  600, ...,  100,  100,  100],
       [ 550,  550,  550, ..., 1000, 1000, 1000],
       [ 620,  620,  610, ..., 1025, 1025, 1025],
       ...,
       [   0,    0,    0, ...,    0, 2450,    0],
       [   0,    0,    0, ...,    0, 2500,    0],
       [   0,    0,    0, ...,    0,  950,    0]], dtype=uint16)

In [12]:
adj_Strike = [[Option_Strike[0][0]]]
adj_Maturity = [[Option_Maturity[0][0]]]
adj_Security = [[Security_Strike[0][0]]]
adj_Price = [[Option_Price[0][0]]]
a = 0
maxit = 6938
for i in range(maxit):
    k = i
    old, old_mat, flag, j, pivot = 0, 0, 0, 1, 1
    if i>0:
        adj_Strike.append([Option_Strike[i][0]])
        adj_Maturity.append([Option_Maturity[i][0]])
        adj_Security.append([Security_Strike[i][0]])
        adj_Price.append([Option_Price[i][0]])
    old = Option_Strike[i][0]
    old_mat = Option_Maturity[i][0]
    Option_Strike[i][0] = -1
    while j < 981:
        if Option_Strike[k][j] != -1:
            if j>pivot and Option_Strike[k][j] != old:
                if flag==0:
                    pivot = j
                if old_mat > 7:
                    refer = k
                    while k<maxit and (Option_Strike[k][j] != old or old_mat!=Option_Maturity[k][j]+7):
                        k+=1
                        flag = 1
                    if k==maxit:
                        while Option_Strike[refer][j] != old and refer>0:
                            refer-=1
                        if Option_Strike[refer][j] == old:
                            k=refer
                    if k<maxit and old_mat==Option_Maturity[k][j]+7:
                        adj_Strike[a].append(Option_Strike[k][j])
                        adj_Maturity[a].append(Option_Maturity[k][j])
                        adj_Security[a].append(Security_Strike[k][j])
                        adj_Price[a].append(Option_Price[k][j])
                    else:
                        if len(adj_Maturity[a])<5 or 65535 in adj_Strike[a]:
                            adj_Maturity.pop(a)
                            adj_Strike.pop(a)
                            adj_Security.pop(a)
                            adj_Price.pop(a)
                        else:
                            a+=1
                        if flag==1:
                            flag=0
                            j = pivot
                            k = i
                            adj_Strike.append([Option_Strike[k][j]])
                            adj_Maturity.append([Option_Maturity[k][j]])
                            adj_Security.append([Security_Strike[k][j]])
                            adj_Price.append([Option_Price[k][j]])
                        else:
                            adj_Strike.append([Option_Strike[k][j]])
                            adj_Maturity.append([Option_Maturity[k][j]])
                            adj_Security.append([Security_Strike[k][j]])
                            adj_Price.append([Option_Price[k][j]])
                            adj_Strike[a].append(Option_Strike[k][j])
                            adj_Maturity[a].append(Option_Maturity[k][j])
                            adj_Security[a].append(Security_Strike[k][j])
                            adj_Price[a].append(Option_Price[k][j])
                else:
                    if len(adj_Maturity[a])<5 or 65535 in adj_Strike[a]:
                        adj_Maturity.pop(a)
                        adj_Strike.pop(a)
                        adj_Security.pop(a)
                        adj_Price.pop(a)
                    else:
                        a+=1
                    if flag==1:
                        flag=0
                        j = pivot
                        k = i
                        adj_Strike.append([Option_Strike[k][j]])
                        adj_Maturity.append([Option_Maturity[k][j]])
                        adj_Security.append([Security_Strike[k][j]])
                        adj_Price.append([Option_Price[k][j]])
                    else:
                        adj_Strike.append([Option_Strike[k][j]])
                        adj_Maturity.append([Option_Maturity[k][j]])
                        adj_Security.append([Security_Strike[k][j]])
                        adj_Price.append([Option_Price[k][j]])
                        adj_Strike[a].append(Option_Strike[k][j])
                        adj_Maturity[a].append(Option_Maturity[k][j])
                        adj_Security[a].append(Security_Strike[k][j])
                        adj_Price[a].append(Option_Price[k][j])
            else:
                if (old_mat > 7 or j==1) and old_mat==Option_Maturity[k][j]+7:
                    adj_Strike[a].append(Option_Strike[k][j])
                    adj_Maturity[a].append(Option_Maturity[k][j])
                    adj_Security[a].append(Security_Strike[k][j])
                    adj_Price[a].append(Option_Price[k][j])
                else:
                    if len(adj_Maturity[a])<5 or 65535 in adj_Strike[a]:
                        adj_Maturity.pop(a)
                        adj_Strike.pop(a)
                        adj_Security.pop(a)
                        adj_Price.pop(a)
                    else:
                        a+=1
                    if flag==1:
                        flag=0
                        j = pivot
                        k = i
                        adj_Strike.append([Option_Strike[k][j]])
                        adj_Maturity.append([Option_Maturity[k][j]])
                        adj_Security.append([Security_Strike[k][j]])
                        adj_Price.append([Option_Price[k][j]])
                    else:
                        adj_Strike.append([Option_Strike[k][j]])
                        adj_Maturity.append([Option_Maturity[k][j]])
                        adj_Security.append([Security_Strike[k][j]])
                        adj_Price.append([Option_Price[k][j]])
                        adj_Strike[a].append(Option_Strike[k][j])
                        adj_Maturity[a].append(Option_Maturity[k][j])
                        adj_Security[a].append(Security_Strike[k][j])
                        adj_Price[a].append(Option_Price[k][j])
            old = Option_Strike[k][j]
            old_mat = Option_Maturity[k][j]
            Option_Strike[k][j] = -1
            if (j==980 and flag==1) or (k==maxit-1 and flag==1):
                if len(adj_Maturity[a])<5 or 65535 in adj_Strike[a]:
                    adj_Maturity.pop(a)
                    adj_Strike.pop(a)
                    adj_Security.pop(a)
                    adj_Price.pop(a)
                else:
                    a+=1
                if flag==1:
                    flag=0
                    j = pivot
                    k = i
                    adj_Strike.append([Option_Strike[k][j]])
                    adj_Maturity.append([Option_Maturity[k][j]])
                    adj_Security.append([Security_Strike[k][j]])
                    adj_Price.append([Option_Price[k][j]])
                    Option_Strike[k][j] = -1
                else:
                    adj_Strike.append([Option_Strike[k][j]])
                    adj_Maturity.append([Option_Maturity[k][j]])
                    adj_Security.append([Security_Strike[k][j]])
                    adj_Price.append([Option_Price[k][j]])
                    Option_Strike[k][j] = -1
            else:
                j+=1
        else:
            j+=1
    if len(adj_Maturity[a])<5 or 65535 in adj_Strike[a]:
        adj_Maturity.pop(a)
        adj_Strike.pop(a)
        adj_Security.pop(a)
        adj_Price.pop(a)
    else:
        a+=1

In [13]:
for i in range(len(adj_Maturity)):
    if len(adj_Maturity[i])>1:
        if adj_Maturity[i][0]==adj_Maturity[i][1]:
            adj_Maturity[i].pop(0)
            adj_Strike[i].pop(0)
            adj_Security[i].pop(0)
            adj_Price[i].pop(0)

In [14]:
a

79089

In [15]:
adj_Strike_df = pd.DataFrame(adj_Strike).fillna(0).astype(int)
adj_Strike_df.to_csv('C:/Users/aless/Documents/Paper/Strike_adj.csv')

In [16]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(adj_Strike_df)

In [17]:
adj_Maturity_df = pd.DataFrame(adj_Maturity).fillna(0).astype(int)
adj_Maturity_df.to_csv('C:/Users/aless/Documents/Paper/Maturity_adj.csv')

In [18]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(adj_Maturity_df)

In [19]:
adj_Security_df = pd.DataFrame(adj_Security).fillna(0).astype(int)
adj_Security_df.to_csv('C:/Users/aless/Documents/Paper/Security_adj.csv')

In [20]:
adj_Price_df = pd.DataFrame(adj_Price).fillna(0).astype(int)
adj_Price_df.to_csv('C:/Users/aless/Documents/Paper/Price_adj.csv')